In [47]:
import pandas
import pandas as pd

from GA import GA
import optproblems.cec2005 as optproblems

# Test Functions
- F2: Shifted Schwefel’s Problem 1.2
    - Uni-modal without noise
- F4: Shifted Schwefel’s Problem 1.2 with Noise in Fitness
    - Uni-modal with noise
- F8: Shifted Rotated Ackley’s Function with Global Optimum on Bounds
    - Single function
    - Global optimum on bounds
    - Multi-modal
- F13: Shifted Expanded Griewank’s plus Rosenbrock’s Function (F8F2)
    - Expanded Function
    - Multi-modal
- F17: Rotated Version of Hybrid Composition Function with Noise in Fitness
    - Gaussian noise in fitness
    - Multi-modal
    - Huge number of local optima
    - In-deterministic
- F24: Rotated Hybrid Composition Function
    - Composite function
    - Huge number of local optima
    - Multi-modal

# 1st Iteration of Configurations - Experiment with GA with Different Hyper Parameters
First we explore how GA evaluates with a set of configurations using only lower numbers of generations

Hyper parameters

In [48]:
# F2 :   Unimodal without noise
# F4 :   Unimodal with noise
# F8 :   Single Function, Global Optimum on Bounds, Multi-modal
# F13:   Expanded Function, Multi-modal
# F17:   Gaussian noise in Fitness, Multi-modal, Huge number of local optima, indeterministic
# F24:   Composite Function, Huge number of local optima, Multi-modal

benchmarks_and_bounds = [optproblems.F2, [-100,100], "F2", optproblems.F4, [-100,100], "F4", optproblems.F8, [-32,32], "F8", optproblems.F13, [-3,1], "F13", optproblems.F17, [-5,5], "F17", optproblems.F24, [-5,5], "F24"]
no_dimensions = [10, 30, 50]
pop_sizes = [5, 10, 15]
num_generations = [50, 150]
tournament_t = [2, 4]
crossover_percentage = [0.5, 0.95, 1]
mutation_rates = [0.2, 0.8, 1]
decreasing_mutation_rate = [True, False]
elites = [0, 1]

average_over_range = 5

Run configuration suite over each configuration combination.

In [49]:
run_configuration_suite = False
if run_configuration_suite:

    list_for_df = []
    for i in range(0, len(benchmarks_and_bounds), 3):
        number_of_functions = (len(benchmarks_and_bounds)/3)
        print(f"Progress: {i/3} / {number_of_functions}")
        bound = benchmarks_and_bounds[i+1]
        benchmark_name = benchmarks_and_bounds[i+2]
        for no_dimension in no_dimensions:
            print(f"\tDimension: {no_dimension}")
            benchmark = benchmarks_and_bounds[i](no_dimension)
            optimal_solution = benchmark.get_optimal_solutions()
            benchmark.evaluate(optimal_solution[0])
            optimal_minimum = optimal_solution[0].objective_values
            for pop_size in pop_sizes:
                print(f"\t\tPopulation: {pop_size}")
                for t in tournament_t:
                    for crossover in crossover_percentage:
                        for mutation in mutation_rates:
                            for dmr in decreasing_mutation_rate:
                                for elite in elites:
                                    for generations in num_generations:
                                        average_fitness = 0
                                        average_difference_from_optimal = 0

                                        best_fitness = 0
                                        best_time = 0
                                        best_difference_from_optimal = 0

                                        for j in range(average_over_range):
                                            algorithm = GA(benchmark, no_dimension, bound, pop_size, num_generations=generations, t=t, crossover_percentage=crossover, mutation_rate=mutation, decreasing_mutation_rate=dmr, elite=elite)
                                            result, time = algorithm.run()
                                            fitness = result.fitness
                                            diff_from_optimal = fitness - optimal_minimum

                                            average_fitness += 1/average_over_range * fitness
                                            average_difference_from_optimal += 1/average_over_range * diff_from_optimal

                                            if j == 0:
                                                best_fitness = fitness
                                                best_difference_from_optimal = diff_from_optimal
                                                best_time = time
                                                continue

                                            if fitness < best_fitness:
                                                best_fitness = fitness

                                            if diff_from_optimal < best_difference_from_optimal:
                                                best_difference_from_optimal = diff_from_optimal

                                            if time < best_time:
                                                best_time = time

                                        result_and_configuration = [benchmark_name, optimal_minimum, best_fitness, best_difference_from_optimal, average_fitness, average_difference_from_optimal, best_time, generations, no_dimension, pop_size, t, crossover, mutation, dmr, elite, bound, average_over_range]
                                        list_for_df.append(result_and_configuration)

    labels = ["TestFunction", "OptimalMinimum", "BestFitness", "BestDiffFromOpt", "AverageFitness", "AverageDiffFromOpt", "TimePerRun", "Generations", "NoDimension", "PopSize", "TForTournament", "CrossoverPerc", "MutationRate", "DecreasingMutationRate", "Elite", "Bounds", "AverageOverRange"]
    df = pandas.DataFrame(data=list_for_df, columns=labels)
    df.to_pickle("GA_df.pkl")
else:
    df = pd.read_pickle("GA_df.pkl")

## Results of the 1st Iteration

# F2:

- Best Diff: 312.11 -> 150 Generation / 10 Dimension / 15 Pop Size / 2 T / 100% CR / 20% MR / DMR True / 1 Elite
    -  Error: 312.11 / 450 -> 69.36%
- Avr. Diff: 913.20 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 95%  CR / 20% MR / DMR True / 1 Elite
    - Error: 913.20 / 450 -> 202.93%

In [50]:
df.query("TestFunction == 'F2'")

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
0,F2,-450.0,17690.525955,18140.525955,3.213329e+04,3.258329e+04,0.011128,50,10,5,2,0.5,0.2,True,0,"[-100, 100]",5
1,F2,-450.0,14235.020676,14685.020676,2.238459e+04,2.283459e+04,0.027806,150,10,5,2,0.5,0.2,True,0,"[-100, 100]",5
2,F2,-450.0,6013.072511,6463.072511,1.487255e+04,1.532255e+04,0.010348,50,10,5,2,0.5,0.2,True,1,"[-100, 100]",5
3,F2,-450.0,1868.742268,2318.742268,5.054819e+03,5.504819e+03,0.031108,150,10,5,2,0.5,0.2,True,1,"[-100, 100]",5
4,F2,-450.0,21611.965380,22061.965380,5.134586e+04,5.179586e+04,0.009959,50,10,5,2,0.5,0.2,False,0,"[-100, 100]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,F2,-450.0,112341.304928,112791.304928,1.461942e+05,1.466442e+05,0.321514,150,50,15,4,1.0,1.0,True,1,"[-100, 100]",5
1292,F2,-450.0,590805.599163,591255.599163,9.641332e+05,9.645832e+05,0.171331,50,50,15,4,1.0,1.0,False,0,"[-100, 100]",5
1293,F2,-450.0,575018.988130,575468.988130,1.290336e+06,1.290786e+06,0.513789,150,50,15,4,1.0,1.0,False,0,"[-100, 100]",5
1294,F2,-450.0,232243.327721,232693.327721,3.293918e+05,3.298418e+05,0.154155,50,50,15,4,1.0,1.0,False,1,"[-100, 100]",5


# F4
- Best Diff: 459.44 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 95% CR / 20% MR / DMR False / 1 Elite
    - Error: 459.44 / 450 ->  102.09%
- Avr. Diff: 1841.4 -> 150 Generation / 10 Dimension / 15 Pop Size / 2 T / 95%  CR / 20% MR / DMR True / 1 Elite
    - Error: 1841.4 / 450 ->  409.2%

In [51]:
df.query("TestFunction == 'F4'")

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
1296,F4,-450.0,19648.328195,20098.328195,4.668801e+04,4.713801e+04,0.009338,50,10,5,2,0.5,0.2,True,0,"[-100, 100]",5
1297,F4,-450.0,43889.355029,44339.355029,5.329062e+04,5.374062e+04,0.028457,150,10,5,2,0.5,0.2,True,0,"[-100, 100]",5
1298,F4,-450.0,3226.647401,3676.647401,1.438899e+04,1.483899e+04,0.009620,50,10,5,2,0.5,0.2,True,1,"[-100, 100]",5
1299,F4,-450.0,1952.865879,2402.865879,6.795449e+03,7.245449e+03,0.032871,150,10,5,2,0.5,0.2,True,1,"[-100, 100]",5
1300,F4,-450.0,53798.825239,54248.825239,1.240192e+05,1.244692e+05,0.011265,50,10,5,2,0.5,0.2,False,0,"[-100, 100]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2587,F4,-450.0,161980.060540,162430.060540,2.346768e+05,2.351268e+05,0.328651,150,50,15,4,1.0,1.0,True,1,"[-100, 100]",5
2588,F4,-450.0,404325.050245,404775.050245,1.207540e+06,1.207990e+06,0.172086,50,50,15,4,1.0,1.0,False,0,"[-100, 100]",5
2589,F4,-450.0,474857.028769,475307.028769,1.385936e+06,1.386386e+06,0.524369,150,50,15,4,1.0,1.0,False,0,"[-100, 100]",5
2590,F4,-450.0,367568.143286,368018.143286,5.447924e+05,5.452424e+05,0.155512,50,50,15,4,1.0,1.0,False,1,"[-100, 100]",5


# F8
- Best Diff: 20.17 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 100% CR / 80% MR / DMR True / 1 Elite
    - Error: 20.17 / 140 -> 14.4%
- Avr. Diff: 20.47 -> Same
    - Error: 20.47 / 140 -> 14.6%

In [52]:
df.query("TestFunction == 'F8'")

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
2592,F8,-140.0,-118.828332,21.171668,-118.601792,21.398208,0.011346,50,10,5,2,0.5,0.2,True,0,"[-32, 32]",5
2593,F8,-140.0,-118.780497,21.219503,-118.632335,21.367665,0.034004,150,10,5,2,0.5,0.2,True,0,"[-32, 32]",5
2594,F8,-140.0,-119.053467,20.946533,-118.934699,21.065301,0.015518,50,10,5,2,0.5,0.2,True,1,"[-32, 32]",5
2595,F8,-140.0,-119.263744,20.736256,-119.165199,20.834801,0.046569,150,10,5,2,0.5,0.2,True,1,"[-32, 32]",5
2596,F8,-140.0,-118.832553,21.167447,-118.571401,21.428599,0.017156,50,10,5,2,0.5,0.2,False,0,"[-32, 32]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3883,F8,-140.0,-118.752831,21.247169,-118.683254,21.316746,0.383418,150,50,15,4,1.0,1.0,True,1,"[-32, 32]",5
3884,F8,-140.0,-118.536631,21.463369,-118.443720,21.556280,0.194784,50,50,15,4,1.0,1.0,False,0,"[-32, 32]",5
3885,F8,-140.0,-118.496857,21.503143,-118.436180,21.563820,0.586340,150,50,15,4,1.0,1.0,False,0,"[-32, 32]",5
3886,F8,-140.0,-118.684637,21.315363,-118.661321,21.338679,0.175834,50,50,15,4,1.0,1.0,False,1,"[-32, 32]",5


# F13
- Best Diff: 1.8799 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 95% CR / 20% MR / DMR True / 0 Elite
    - Error: 1.8799 / 130 ->  1.34%
- Avr. Diff: 2.9072 -> 150 Generation / 10 Dimension / 10 Pop Size / 2 T / 95% CR / 20% MR / DMR True / 1 Elite
    - Error: 2.9072 / 130 ->  2.28%


In [53]:
df.query("TestFunction == 'F13'")

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
0,F13,-130.0,-114.747967,15.252033,123.382008,253.382008,0.015493,50,10,5,2,0.5,0.2,True,0,"[-3, 1]",5
1,F13,-130.0,-112.484813,17.515187,-81.314126,48.685874,0.047154,150,10,5,2,0.5,0.2,True,0,"[-3, 1]",5
2,F13,-130.0,-119.482405,10.517595,-117.122011,12.877989,0.014168,50,10,5,2,0.5,0.2,True,1,"[-3, 1]",5
3,F13,-130.0,-127.111143,2.888857,-122.551732,7.448268,0.043210,150,10,5,2,0.5,0.2,True,1,"[-3, 1]",5
4,F13,-130.0,68.014676,198.014676,1145.731194,1275.731194,0.015538,50,10,5,2,0.5,0.2,False,0,"[-3, 1]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,F13,-130.0,18.446939,148.446939,86.802009,216.802009,0.559082,150,50,15,4,1.0,1.0,True,1,"[-3, 1]",5
1292,F13,-130.0,3078.646003,3208.646003,5533.162406,5663.162406,0.246466,50,50,15,4,1.0,1.0,False,0,"[-3, 1]",5
1293,F13,-130.0,5464.382972,5594.382972,9402.327325,9532.327325,0.757659,150,50,15,4,1.0,1.0,False,0,"[-3, 1]",5
1294,F13,-130.0,1636.088684,1766.088684,2376.984034,2506.984034,0.231453,50,50,15,4,1.0,1.0,False,1,"[-3, 1]",5


# F17
- Best Diff: 219.97 -> 150 Generation / 10 Dimension / 10 Pop Size / 4 T / 50% CR / 80% MR / DMR True / 1 Elite
    - Error: 219.97 / 120 ->  183.3%
- Avr. Diff: 283.42 -> 150 Generation / 10 Dimension / 15 Pop Size / 2 T / 100% CR / 100% MR / DMR True / 1 Elite
    - Error: 283.42 / 120 ->  236.2%

In [54]:
df.query("TestFunction == 'F17'")

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
1296,F17,120.0,748.766137,628.766137,1079.084525,959.084525,0.113645,50,10,5,2,0.5,0.2,True,0,"[-5, 5]",5
1297,F17,120.0,591.077512,471.077512,951.976583,831.976583,0.335133,150,10,5,2,0.5,0.2,True,0,"[-5, 5]",5
1298,F17,120.0,454.579857,334.579857,557.498490,437.498490,0.110451,50,10,5,2,0.5,0.2,True,1,"[-5, 5]",5
1299,F17,120.0,403.606848,283.606848,495.791958,375.791958,0.332959,150,10,5,2,0.5,0.2,True,1,"[-5, 5]",5
1300,F17,120.0,1369.054293,1249.054293,1429.394142,1309.394142,0.110137,50,10,5,2,0.5,0.2,False,0,"[-5, 5]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2587,F17,120.0,692.699934,572.699934,825.222940,705.222940,4.463455,150,50,15,4,1.0,1.0,True,1,"[-5, 5]",5
2588,F17,120.0,1726.874128,1606.874128,1875.775029,1755.775029,1.468171,50,50,15,4,1.0,1.0,False,0,"[-5, 5]",5
2589,F17,120.0,1809.362841,1689.362841,1871.932725,1751.932725,4.641679,150,50,15,4,1.0,1.0,False,0,"[-5, 5]",5
2590,F17,120.0,1286.370225,1166.370225,1564.226187,1444.226187,1.467616,50,50,15,4,1.0,1.0,False,1,"[-5, 5]",5


# F24
- Best Diff: 500.05 -> 150 Generation / 10 Dimension / 15 Pop Size / 2 T / 100% CR / 20% MR / DMR False / 1 Elite
    - Error: 500.05 / 260 -> 192.32%
- Avr. Diff: 836.26 -> 150 Generation / 10 Dimension / 10 Pop Size / 4 T / 100% CR / 80% MR / DMR True / 1 Elite
    - Error: 835.26 / 260 -> 321.25%

In [55]:
df.query("TestFunction == 'F24'")

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
0,F24,260.0,1581.379194,1321.379194,1772.169403,1512.169403,0.115887,50,10,5,2,0.5,0.2,True,0,"[-5, 5]",5
1,F24,260.0,1716.297855,1456.297855,1859.331639,1599.331639,0.334574,150,10,5,2,0.5,0.2,True,0,"[-5, 5]",5
2,F24,260.0,1257.358857,997.358857,1496.301694,1236.301694,0.095941,50,10,5,2,0.5,0.2,True,1,"[-5, 5]",5
3,F24,260.0,1009.196942,749.196942,1260.036581,1000.036581,0.290794,150,10,5,2,0.5,0.2,True,1,"[-5, 5]",5
4,F24,260.0,1706.802306,1446.802306,1905.026391,1645.026391,0.097299,50,10,5,2,0.5,0.2,False,0,"[-5, 5]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,F24,260.0,1604.662489,1344.662489,1633.768568,1373.768568,3.663990,150,50,15,4,1.0,1.0,True,1,"[-5, 5]",5
1292,F24,260.0,2013.695892,1753.695892,2032.972450,1772.972450,1.252370,50,50,15,4,1.0,1.0,False,0,"[-5, 5]",5
1293,F24,260.0,2022.287128,1762.287128,2078.779992,1818.779992,3.872165,150,50,15,4,1.0,1.0,False,0,"[-5, 5]",5
1294,F24,260.0,1880.468349,1620.468349,1907.107345,1647.107345,1.220936,50,50,15,4,1.0,1.0,False,1,"[-5, 5]",5


# Conclusion:

The optimizer performed the best on F8 and F13. F13 error was the lowest with the best fitness 1.34% away from the optimal. F8's best fitness was 14.4% away from the optimal. While the F13 error is good F8 is a significantly larger from the optimal and can be approved. The GA performed poorly on the rest of the test functions. The best fitness errors are as follows:
- F2:  69.36%
- F4: 102.09%
- F17: 183.3%
- F24: 192.32%

Each test function scored the best after 150 generations, so we decided to increase the number of generations to see how they improve. For the 2nd iteration we kept the configuration of the best algorithm settings. The configurations with low frequency from the bests weren't carried on to the 2nd iteration for the algorithms where the error was huge. However, F13 scored the best fitness without elitism and this was the only configuration among the best configurations to not use elitism. For this configuration option was carried on to the 2nd iteration.

Frequency of configuration among best difference and average difference from optimal:
- Generation:
    - 50 -> 0
    - 150 -> 12
- Dimension:
    - 10 -> 12
    - 30 -> 0
    - 50 -> 0
- Population Size:
    - 5 -> 0
    - 10 -> 3
    - 15 -> 9
- Tournament Size:
    - 2 -> 5
    - 4 -> 7
- Crossover Rate:
    - 50% -> 1
    - 95% -> 5
    - 100% -> 6
- Mutation Rate:
    - 50% -> 1
    - 95% -> 5
    - 100% -> 6
- Decreasing Mutation Rate:
    - True -> 10
    - False -> 2
- Elites:
    - 0 -> 1
    - 1 -> 11

# 2nd Iteration of Configurations
Same algorithm was used as for the 1st iteration but different hyper-parameters. Increased the number of generations to 250, 500, 1000. The population size was increased too and some configurations weren't carried over from iteration 1, i.e. the number of dimension was reduced to only 10.

In [56]:
benchmarks_and_bounds = [optproblems.F2, [-100,100], "F2", optproblems.F4, [-100,100], "F4", optproblems.F8, [-32,32], "F8", optproblems.F13, [-3,1], "F13", optproblems.F17, [-5,5], "F17", optproblems.F24, [-5,5], "F24"]
no_dimensions = [10]
num_generations = [250, 500, 1000]
pop_sizes = [10, 15, 20]
tournament_t = [2, 4]
crossover_percentage = [0.95, 1]
mutation_rates = [0.2, 0.8]
decreasing_mutation_rate = [True]
elites = [0, 1]

average_over_range = 5

In [57]:
run_configuration_suite = False
if run_configuration_suite:

    list_for_df = []
    for i in range(0, len(benchmarks_and_bounds), 3):
        number_of_functions = (len(benchmarks_and_bounds)/3)
        print(f"Progress: {i/3} / {number_of_functions}")
        bound = benchmarks_and_bounds[i+1]
        benchmark_name = benchmarks_and_bounds[i+2]
        for no_dimension in no_dimensions:
            print(f"\tDimension: {no_dimension}")
            benchmark = benchmarks_and_bounds[i](no_dimension)
            optimal_solution = benchmark.get_optimal_solutions()
            benchmark.evaluate(optimal_solution[0])
            optimal_minimum = optimal_solution[0].objective_values
            for pop_size in pop_sizes:
                print(f"\t\tPopulation: {pop_size}")
                for t in tournament_t:
                    for crossover in crossover_percentage:
                        for mutation in mutation_rates:
                            for dmr in decreasing_mutation_rate:
                                for elite in elites:
                                    for generations in num_generations:
                                        average_fitness = 0
                                        average_difference_from_optimal = 0

                                        best_fitness = 0
                                        best_time = 0
                                        best_difference_from_optimal = 0

                                        for j in range(average_over_range):
                                            algorithm = GA(benchmark, no_dimension, bound, pop_size, num_generations=generations, t=t, crossover_percentage=crossover, mutation_rate=mutation, decreasing_mutation_rate=dmr, elite=elite)
                                            result, time = algorithm.run()
                                            fitness = result.fitness
                                            diff_from_optimal = fitness - optimal_minimum

                                            average_fitness += 1/average_over_range * fitness
                                            average_difference_from_optimal += 1/average_over_range * diff_from_optimal

                                            if j == 0:
                                                best_fitness = fitness
                                                best_difference_from_optimal = diff_from_optimal
                                                best_time = time
                                                continue

                                            if fitness < best_fitness:
                                                best_fitness = fitness

                                            if diff_from_optimal < best_difference_from_optimal:
                                                best_difference_from_optimal = diff_from_optimal

                                            if time < best_time:
                                                best_time = time

                                        result_and_configuration = [benchmark_name, optimal_minimum, best_fitness, best_difference_from_optimal, average_fitness, average_difference_from_optimal, best_time, generations, no_dimension, pop_size, t, crossover, mutation, dmr, elite, bound, average_over_range]
                                        list_for_df.append(result_and_configuration)

    labels = ["TestFunction", "OptimalMinimum", "BestFitness", "BestDiffFromOpt", "AverageFitness", "AverageDiffFromOpt", "TimePerRun", "Generations", "NoDimension", "PopSize", "TForTournament", "CrossoverPerc", "MutationRate", "DecreasingMutationRate", "Elite", "Bounds", "AverageOverRange"]
    df_II = pandas.DataFrame(data=list_for_df, columns=labels)
    df_II.to_pickle("GA_df_specific.pkl")
else:
    df_II = pd.read_pickle("GA_df_specific.pkl")

# Results of the 2nd Iteration

# F2
### For 1st Iteration:
- Best Diff: 312.11 -> 150 Generation / 10 Dimension / 15 Pop Size / 2 T / 100% CR / 20% MR / DMR True / 1 Elite
    -  Error: 312.11 / 450 -> 69.36%
- Avr. Diff: 913.20 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 95%  CR / 20% MR / DMR True / 1 Elite
    - Error: 913.20 / 450 -> 202.93%

### For 2nd Iteration:
- Best Diff: 33.482 -> 1000 Generation / 10 Dimension / 20 Pop Size / 4 T / 95% CR / 20% MR / DMR True / 0 Elite
    -  Error: 33.482 / 450 -> 7.4%
- Avr. Diff: 113.75 -> 1000 Generation / 10 Dimension / 20 Pop Size / 4 T / 95% CR / 20% MR / DMR True / 1 Elite
    - Error: 113.75 / 450 -> 25.27%

## Conclusion for F2:
- Significant improvement by increasing the generations from 150 -> 1000 and the population from 15 -> 20.
- Best diff error is ~10× less
- Avg diff error is ~8× less
- In the 2nd iteration the best fitness was achieved by 4 as tournament size and 95% for crossover rate instead of 2 and 100%.

In [66]:
df_II.query("TestFunction == 'F2'")

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
0,F2,-450.0,3553.283953,4003.283953,10513.100009,10963.100009,0.099280,250,10,10,2,0.95,0.2,True,0,"[-100, 100]",5
1,F2,-450.0,3355.970747,3805.970747,6196.958038,6646.958038,0.173902,500,10,10,2,0.95,0.2,True,0,"[-100, 100]",5
2,F2,-450.0,1849.765035,2299.765035,4120.509517,4570.509517,0.334554,1000,10,10,2,0.95,0.2,True,0,"[-100, 100]",5
3,F2,-450.0,369.467035,819.467035,1330.410519,1780.410519,0.097897,250,10,10,2,0.95,0.2,True,1,"[-100, 100]",5
4,F2,-450.0,-5.245785,444.754215,855.020040,1305.020040,0.183190,500,10,10,2,0.95,0.2,True,1,"[-100, 100]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,F2,-450.0,651.537894,1101.537894,1653.836877,2103.836877,0.419007,500,10,20,4,1.00,0.8,True,0,"[-100, 100]",5
140,F2,-450.0,43.057849,493.057849,843.527527,1293.527527,0.848008,1000,10,20,4,1.00,0.8,True,0,"[-100, 100]",5
141,F2,-450.0,216.965097,666.965097,630.764040,1080.764040,0.218464,250,10,20,4,1.00,0.8,True,1,"[-100, 100]",5
142,F2,-450.0,215.840259,665.840259,647.389035,1097.389035,0.439477,500,10,20,4,1.00,0.8,True,1,"[-100, 100]",5


# F4
### For 1st Iteration
- Best Diff: 459.44 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 95% CR / 20% MR / DMR False / 1 Elite
    - Error: 459.44 / 450 ->  102.09%
- Avr. Diff: 1841.4 -> 150 Generation / 10 Dimension / 15 Pop Size / 2 T / 95%  CR / 20% MR / DMR True / 1 Elite
    - Error: 1841.4 / 450 ->  409.2%

### For 2nd Iteration:
- Best Diff: 56.65 -> 500 Generation / 10 Dimension / 15 Pop Size / 4 T / 95% CR / 20% MR / DMR True / 1 Elite
    - Error: 56.65 / 450 -> 12.58%
- Avr. Diff: 190.78 -> 1000 Generation / 10 Dimension / 20 Pop Size / 4 T / 100% CR / 20% MR / DMR True / 1 Elite
    - Error: 190.78 / 450 -> 42.39%

## Conclusion for F4:
- Significant change by increasing the generation.
- The best fitness was achieved with 500 generation instead of 1000 and the population size was same for both iterations.
- The best fitness parameters didn't change much. Decreasing MR was used and the generation was increased from 150 to 500. However, for the average fitness the parameters changed. Higher population (20), generation (1000), tournament size (4) and crossover rate (100%) was used.

In [59]:
df_II.query("TestFunction == 'F4'")

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
144,F4,-450.0,5466.510035,5916.510035,10096.430857,10546.430857,0.088707,250,10,10,2,0.95,0.2,True,0,"[-100, 100]",5
145,F4,-450.0,2410.322413,2860.322413,5507.949389,5957.949389,0.178253,500,10,10,2,0.95,0.2,True,0,"[-100, 100]",5
146,F4,-450.0,1527.051754,1977.051754,3311.220261,3761.220261,0.371345,1000,10,10,2,0.95,0.2,True,0,"[-100, 100]",5
147,F4,-450.0,255.286860,705.286860,1985.977053,2435.977053,0.094318,250,10,10,2,0.95,0.2,True,1,"[-100, 100]",5
148,F4,-450.0,326.773764,776.773764,1059.221200,1509.221200,0.192249,500,10,10,2,0.95,0.2,True,1,"[-100, 100]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,F4,-450.0,1038.801267,1488.801267,2028.070794,2478.070794,0.447136,500,10,20,4,1.00,0.8,True,0,"[-100, 100]",5
284,F4,-450.0,-292.184969,157.815031,1072.513053,1522.513053,0.890534,1000,10,20,4,1.00,0.8,True,0,"[-100, 100]",5
285,F4,-450.0,171.422477,621.422477,1092.690191,1542.690191,0.230837,250,10,20,4,1.00,0.8,True,1,"[-100, 100]",5
286,F4,-450.0,-270.407459,179.592541,412.955613,862.955613,0.456779,500,10,20,4,1.00,0.8,True,1,"[-100, 100]",5


In [60]:
df_II.query("TestFunction == 'F8'")

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
288,F8,-140.0,-118.968130,21.031870,-118.857024,21.142976,0.114117,250,10,10,2,0.95,0.2,True,0,"[-32, 32]",5
289,F8,-140.0,-119.185708,20.814292,-119.029558,20.970442,0.210737,500,10,10,2,0.95,0.2,True,0,"[-32, 32]",5
290,F8,-140.0,-119.033946,20.966054,-118.975702,21.024298,0.422029,1000,10,10,2,0.95,0.2,True,0,"[-32, 32]",5
291,F8,-140.0,-119.338976,20.661024,-119.256871,20.743129,0.109634,250,10,10,2,0.95,0.2,True,1,"[-32, 32]",5
292,F8,-140.0,-119.498635,20.501365,-119.373761,20.626239,0.222074,500,10,10,2,0.95,0.2,True,1,"[-32, 32]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,F8,-140.0,-119.153839,20.846161,-119.093613,20.906387,0.506419,500,10,20,4,1.00,0.8,True,0,"[-32, 32]",5
428,F8,-140.0,-119.413861,20.586139,-119.317191,20.682809,1.023721,1000,10,20,4,1.00,0.8,True,0,"[-32, 32]",5
429,F8,-140.0,-119.408291,20.591709,-119.360298,20.639702,0.258451,250,10,20,4,1.00,0.8,True,1,"[-32, 32]",5
430,F8,-140.0,-119.603558,20.396442,-119.500387,20.499613,0.519927,500,10,20,4,1.00,0.8,True,1,"[-32, 32]",5


In [61]:
df_II.query("TestFunction == 'F13'")

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
432,F13,-130.0,-126.943426,3.056574,-124.507586,5.492414,0.138476,250,10,10,2,0.95,0.2,True,0,"[-3, 1]",5
433,F13,-130.0,-128.085228,1.914772,-126.321452,3.678548,0.279909,500,10,10,2,0.95,0.2,True,0,"[-3, 1]",5
434,F13,-130.0,-127.980574,2.019426,-126.913084,3.086916,0.579804,1000,10,10,2,0.95,0.2,True,0,"[-3, 1]",5
435,F13,-130.0,-128.705467,1.294533,-127.684387,2.315613,0.144806,250,10,10,2,0.95,0.2,True,1,"[-3, 1]",5
436,F13,-130.0,-129.053033,0.946967,-128.648096,1.351904,0.289962,500,10,10,2,0.95,0.2,True,1,"[-3, 1]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,F13,-130.0,-128.483333,1.516667,-126.959070,3.040930,0.681661,500,10,20,4,1.00,0.8,True,0,"[-3, 1]",5
572,F13,-130.0,-128.965239,1.034761,-128.415565,1.584435,1.372176,1000,10,20,4,1.00,0.8,True,0,"[-3, 1]",5
573,F13,-130.0,-128.494934,1.505066,-127.519317,2.480683,0.334248,250,10,20,4,1.00,0.8,True,1,"[-3, 1]",5
574,F13,-130.0,-128.648579,1.351421,-128.374870,1.625130,0.679362,500,10,20,4,1.00,0.8,True,1,"[-3, 1]",5


In [62]:
df_II.query("TestFunction == 'F17'")

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
576,F17,120.0,371.916036,251.916036,425.477220,305.477220,1.087346,250,10,10,2,0.95,0.2,True,0,"[-5, 5]",5
577,F17,120.0,413.938939,293.938939,436.805606,316.805606,2.141470,500,10,10,2,0.95,0.2,True,0,"[-5, 5]",5
578,F17,120.0,364.981052,244.981052,380.661387,260.661387,4.319710,1000,10,10,2,0.95,0.2,True,0,"[-5, 5]",5
579,F17,120.0,351.178913,231.178913,378.818352,258.818352,1.050830,250,10,10,2,0.95,0.2,True,1,"[-5, 5]",5
580,F17,120.0,373.734621,253.734621,406.427029,286.427029,2.155800,500,10,10,2,0.95,0.2,True,1,"[-5, 5]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,F17,120.0,280.068122,160.068122,347.955552,227.955552,4.757954,500,10,20,4,1.00,0.8,True,0,"[-5, 5]",5
716,F17,120.0,325.023540,205.023540,353.633431,233.633431,9.791381,1000,10,20,4,1.00,0.8,True,0,"[-5, 5]",5
717,F17,120.0,350.001728,230.001728,367.151403,247.151403,2.336803,250,10,20,4,1.00,0.8,True,1,"[-5, 5]",5
718,F17,120.0,303.276890,183.276890,336.273628,216.273628,4.840253,500,10,20,4,1.00,0.8,True,1,"[-5, 5]",5


In [63]:
df_II.query("TestFunction == 'F24'")

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
720,F24,260.0,991.810296,731.810296,1311.970832,1051.970832,1.042491,250,10,10,2,0.95,0.2,True,0,"[-5, 5]",5
721,F24,260.0,753.377338,493.377338,1033.482308,773.482308,2.107731,500,10,10,2,0.95,0.2,True,0,"[-5, 5]",5
722,F24,260.0,624.964571,364.964571,725.697857,465.697857,4.346395,1000,10,10,2,0.95,0.2,True,0,"[-5, 5]",5
723,F24,260.0,495.428966,235.428966,694.793545,434.793545,1.051611,250,10,10,2,0.95,0.2,True,1,"[-5, 5]",5
724,F24,260.0,470.913009,210.913009,571.767420,311.767420,2.198264,500,10,10,2,0.95,0.2,True,1,"[-5, 5]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,F24,260.0,624.117583,364.117583,860.867563,600.867563,3.955479,500,10,20,4,1.00,0.8,True,0,"[-5, 5]",5
860,F24,260.0,497.332193,237.332193,590.451815,330.451815,8.117383,1000,10,20,4,1.00,0.8,True,0,"[-5, 5]",5
861,F24,260.0,569.282428,309.282428,639.205060,379.205060,2.063875,250,10,20,4,1.00,0.8,True,1,"[-5, 5]",5
862,F24,260.0,500.733891,240.733891,515.011661,255.011661,4.066582,500,10,20,4,1.00,0.8,True,1,"[-5, 5]",5
